In [58]:
from sklearn import datasets
import pandas as pd
import numpy as np
import math as m

In [59]:
def entropy(y):
    values = set(y)
    total_num = len(y)
    entropy = 0
    for i in values:
        #finding the number of values of each class
        p = (y == i).sum() / total_num
        
        #summation formula for total entropy
        entropy += -p*m.log2(p)
        
    return entropy

In [60]:
#calculating information gain if we split upon the feature 'feature'
def info_gain(x , y , feature):
    
    #calculating original information required of parent node
    info_orig = entropy(y)
    
    #using step, we test for various boundaries inside the data, and choose the one with the best information gain
    step = x[x.columns[feature]].mean()/4
    init = step
    
    #initializing key variables
    boundary = 0
    entropy_max = -1000
    split_info_max = -100
    
    #Running a loop, test for info gain for various boundaries
    #Can't use for loop, because for loop cant handle floats well
    while step <= x[x.columns[feature]].max() :
        final_entropy = 0
        y_greater  = y.loc[x[x.columns[feature]] >= step]
        y_less  = y.loc[x[x.columns[feature]] < step]
        
        #coefficients for calculating final entropy
        d1 = len(y_greater)
        d2 = len(y_less)
        d = d1 + d2
        
        #Running the loop to find the perfect boundary and corresponding entropy
        final_entropy = (-d1/d)*(entropy(y_greater)) + (-d2/d)*(entropy(y_less))
        
        #if the resulting entropy is greater(better) for this boundary value as compared to previous boundary value, then
        #then, entropy_max, boundary and split_info_max will store information about this split
        if final_entropy > entropy_max: #the values only get updated if this split performs better than the last one
            entropy_max = final_entropy
            boundary = step
            split_info_max = split_info(y_greater , y_less) 
            
        #finally,we increment step to move on to the next boundary value to test
        #if the next boundary value performs worse than this one, then we keep the this better set of values with us, ie the values don't get updated
        step = step + init

    return info_orig-entropy_max, boundary, split_info_max

In [61]:
def split_info(y1 , y2) :
    #calculating split info for a particular split, y1 and y2 being the two results of the split
    d1 = len(y1)
    d2 = len(y2)
    
    #d is the total number of values in the parent node, the number of values before the split
    d = d1 + d2
    if(d1 == 0 or d2 == 0 or d == 0):
        return 0
    
    #formula of split info
    p = (-1*d1*m.log2(d1/d))/d +  (-1*d2*m.log2(d2/d))/d
    return p

In [62]:
def gain_ratio(x ,y ,feature):
    #formula of the gain ratio
    #information gain divided by split info
    return info_gain(x , y , feature)[0] / info_gain(x , y , feature)[2]

In [63]:
def dt(x , y , features , level):
    
    #Base Case 1: pure node
    #If all the vales in the output series are the same, it implies that it is a leaf node, and it's entropy is zero
    if(len(set(y)) == 1):
        print('Level : ' , level)
        for i in set(y):
            print('Count of', i, ': ', len(y[y == i]))
        print('Current entropy : ' , 0.0)
        print('Reached leaf node')
        print('')
        return
    
    #Base Case 2: all the features have already been used to split, ie all features have been exhausted
    #'features' is the list containing all column names. Once a column is used in splitting, it is removed from the list.
    #An empty 'features' list implies that all the columns have been exhausted for the splitting process.
    if(len(features) == 0):
        print('Level : ' , level)
        for i in set(y):
            print('Count of', i, ': ', len(y[y == i]))
        print('Current entropy : ' , entropy(y))
        print('leaf node reached , no features left')
        print('')
        return
    
    #Initializing max_gain with a really low value
    max_gain = -10000000
    
    #best_feature is the index value for the list containing unused features that can be used to split the data
    best_feature = -1
    
    #boundary is the value along which we'll split the values of the chosen feature. Values will be split as being either greater or lesser than the boundary value.
    boundary = 0
    
    #looping through the list of features to find the best possible feature to split upon, on the basis of the information gain
    for i in features:
        gain = gain_ratio(x , y , i)
        if gain > max_gain:
            max_gain = gain
            best_feature = i
            boundary = info_gain(x , y , i)[1]
    
    features_list.append(best_feature)
    
            
    print('Level : ' , level)
    for i in set(y):
            print('Count of', i, ': ', len(y[y == i]))
    print('Current entropy : ' , entropy(y))
    print('Splitting on x[' , x.columns[best_feature] , '] with gain ratio : ' , max_gain, sep = '')
    print('')

    # finding all possible classes to split upon
    labels = set(x[x.columns[best_feature]])
    
    #removing the used or exhausted feature
    if best_feature not in features:
        return
    features.remove(best_feature)
    
    #the actual splitting of the data
    x_greater  = x.loc[x[x.columns[best_feature]] >= boundary]
    y_greater  = y.loc[x[x.columns[best_feature]] >=  boundary]
    x_lesser  = x.loc[x[x.columns[best_feature]] < boundary]
    y_lesser  = y.loc[x[x.columns[best_feature]] <  boundary]
    
    #recursively calling dt function to print features for further levels
    dt(x_greater , y_greater , features , level + 1)
    dt(x_lesser , y_lesser , features , level + 1)

In [64]:
iris = datasets.load_iris()

#converting the dataset to pandas DataFrame
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],columns= iris['feature_names'] + ['target'])
df.columns = [0, 1, 2, 3, 4]

x = df.iloc[: , 0:-1]
y = df.iloc[: , -1]
list_of_features = x.columns.tolist()

In [65]:
features = [i for i in range(len(x.columns))]

#Calling the main decision tree function
dt(x, y, features , 0)

Level :  0
Count of 0.0 :  50
Count of 1.0 :  50
Count of 2.0 :  50
Current entropy :  1.584962500721156
Splitting on x[1] with gain ratio : 2.9334152819642965

Level :  1
Count of 0.0 :  42
Count of 1.0 :  8
Count of 2.0 :  17
Current entropy :  1.2905041149657768
Splitting on x[0] with gain ratio : 1.8697736472964084

Level :  2
Count of 0.0 :  3
Count of 1.0 :  8
Count of 2.0 :  17
Current entropy :  1.2987207862212027
Splitting on x[3] with gain ratio : 1.9254857336058204

Level :  3
Count of 2.0 :  15
Current entropy :  0.0
Reached leaf node

Level :  3
Count of 0.0 :  3
Count of 1.0 :  8
Count of 2.0 :  2
Current entropy :  1.3346791410515946
Splitting on x[2] with gain ratio : 2.4251091799519626

Level :  4
Count of 1.0 :  8
Count of 2.0 :  2
Current entropy :  0.7219280948873623
leaf node reached , no features left

Level :  4
Count of 0.0 :  3
Current entropy :  0.0
Reached leaf node

Level :  2
Count of 0.0 :  39
Current entropy :  0.0
Reached leaf node

Level :  1
Count of 0